In [1]:
import os
import torch
from utils import get_vocabs
torch.manual_seed(0)

In [2]:
data_dir = 'data_old'
# path_train = os.path.join(data_dir, 'train1_short.wtag')
# path_test = os.path.join(data_dir, 'test1_short.wtag')
path_train = os.path.join(data_dir, 'train.wtag')
path_test = os.path.join(data_dir, 'test.wtag')
paths_list = [path_train, path_test]
word_dict, pos_dict = get_vocabs(paths_list)

In [3]:
from data_handling import PosDataset
from torch.utils.data.dataloader import DataLoader
print("path_train -", path_train)
print("path_test -", path_test)
BATCH_SIZE = 1
paths_list = [path_train, path_test]
word_dict, pos_dict = get_vocabs(paths_list)
train = PosDataset(word_dict, pos_dict, data_dir, 'train', padding=False)
train_dataloader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test = PosDataset(word_dict, pos_dict, data_dir, 'test', padding=False)
test_dataloader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

path_train - data_old/train.wtag
path_test - data_old/test.wtag
idx_pos_mappings - [0, 1, 2, '#', '$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
pos_idx_mappings - {'<pad>': 0, '<unk>': 1, '<root>': 2, '#': 3, '$': 4, "''": 5, ',': 6, '-LRB-': 7, '-RRB-': 8, '.': 9, ':': 10, 'CC': 11, 'CD': 12, 'DT': 13, 'EX': 14, 'FW': 15, 'IN': 16, 'JJ': 17, 'JJR': 18, 'JJS': 19, 'MD': 20, 'NN': 21, 'NNP': 22, 'NNPS': 23, 'NNS': 24, 'PDT': 25, 'POS': 26, 'PRP': 27, 'PRP$': 28, 'RB': 29, 'RBR': 30, 'RBS': 31, 'RP': 32, 'SYM': 33, 'TO': 34, 'UH': 35, 'VB': 36, 'VBD': 37, 'VBG': 38, 'VBN': 39, 'VBP': 40, 'VBZ': 41, 'WDT': 42, 'WP': 43, 'WP$': 44, 'WRB': 45, '``': 46}
idx_pos_mappings - [0, 1, 2, '#', '$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW',

In [4]:
print("Number of Train Tagged Sentences ", len(train))
print("Number of Test Tagged Sentences ",len(test))

Number of Train Tagged Sentences  5000
Number of Test Tagged Sentences  1000


In [5]:
import torch
from torch import nn
import torch.optim as optim
from model import DnnPosTagger
from trainer import Trainer
from loss import NllLoss


HIDDEN_DIM = 100
NUM_LAYERS = 2
word_vocab_size = len(train.word_idx_mappings)
tag_vocab_size = len(train.pos_idx_mappings)

model = DnnPosTagger(train_dataloader.dataset.word_vectors, HIDDEN_DIM, NUM_LAYERS, tag_vocab_size)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

if use_cuda:
    model.cuda()
    
# loss_function = nn.NLLLoss()
loss_function = NllLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
acumulate_grad_steps = 50 # This is the actual batch_size, while we officially use batch_size=1
trainer = Trainer(model, optimizer, loss_function, device)

word embeddings shape: torch.Size([15213, 300])


In [6]:
torch.manual_seed(0)
EPOCHS = 15
acumulate_grad_steps = 50
len_train = len(train)
len_test = len(test)
trainer.train(EPOCHS, train_dataloader, test_dataloader, acumulate_grad_steps, len_train, len_test, early_stopping=5)

tag_scores shape: torch.Size([23, 47])
tag_scores shape: torch.Size([20, 47])
tag_scores shape: torch.Size([55, 47])
tag_scores shape: torch.Size([54, 47])
tag_scores shape: torch.Size([24, 47])
tag_scores shape: torch.Size([11, 47])
tag_scores shape: torch.Size([16, 47])
tag_scores shape: torch.Size([23, 47])
tag_scores shape: torch.Size([29, 47])
tag_scores shape: torch.Size([15, 47])
tag_scores shape: torch.Size([20, 47])
tag_scores shape: torch.Size([9, 47])
tag_scores shape: torch.Size([40, 47])
tag_scores shape: torch.Size([12, 47])
tag_scores shape: torch.Size([29, 47])
tag_scores shape: torch.Size([22, 47])
tag_scores shape: torch.Size([23, 47])
tag_scores shape: torch.Size([6, 47])
tag_scores shape: torch.Size([14, 47])
tag_scores shape: torch.Size([17, 47])
tag_scores shape: torch.Size([15, 47])
tag_scores shape: torch.Size([24, 47])
tag_scores shape: torch.Size([16, 47])
tag_scores shape: torch.Size([38, 47])
tag_scores shape: torch.Size([30, 47])
tag_scores shape: torch.Siz

tag_scores shape: torch.Size([20, 47])
tag_scores shape: torch.Size([19, 47])
tag_scores shape: torch.Size([26, 47])
tag_scores shape: torch.Size([11, 47])
tag_scores shape: torch.Size([17, 47])
tag_scores shape: torch.Size([8, 47])
tag_scores shape: torch.Size([42, 47])
tag_scores shape: torch.Size([44, 47])
tag_scores shape: torch.Size([32, 47])
tag_scores shape: torch.Size([10, 47])
tag_scores shape: torch.Size([34, 47])
tag_scores shape: torch.Size([17, 47])
tag_scores shape: torch.Size([6, 47])
tag_scores shape: torch.Size([24, 47])
tag_scores shape: torch.Size([46, 47])
tag_scores shape: torch.Size([41, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([30, 47])
tag_scores shape: torch.Size([34, 47])
tag_scores shape: torch.Size([8, 47])
tag_scores shape: torch.Size([13, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([31, 47])
tag_scores shape: torch.Size([19, 47])
tag_scores shape: torch.Size([31, 47])
tag_scores shape: torch.Size

tag_scores shape: torch.Size([36, 47])
tag_scores shape: torch.Size([35, 47])
tag_scores shape: torch.Size([33, 47])
tag_scores shape: torch.Size([23, 47])
tag_scores shape: torch.Size([15, 47])
tag_scores shape: torch.Size([34, 47])
tag_scores shape: torch.Size([22, 47])
tag_scores shape: torch.Size([15, 47])
tag_scores shape: torch.Size([4, 47])
tag_scores shape: torch.Size([18, 47])
tag_scores shape: torch.Size([38, 47])
tag_scores shape: torch.Size([39, 47])
tag_scores shape: torch.Size([16, 47])
tag_scores shape: torch.Size([23, 47])
tag_scores shape: torch.Size([34, 47])
tag_scores shape: torch.Size([29, 47])
tag_scores shape: torch.Size([32, 47])
tag_scores shape: torch.Size([28, 47])
tag_scores shape: torch.Size([28, 47])
tag_scores shape: torch.Size([17, 47])
tag_scores shape: torch.Size([19, 47])
tag_scores shape: torch.Size([7, 47])
tag_scores shape: torch.Size([33, 47])
tag_scores shape: torch.Size([33, 47])
tag_scores shape: torch.Size([19, 47])
tag_scores shape: torch.Siz

tag_scores shape: torch.Size([39, 47])
tag_scores shape: torch.Size([40, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([26, 47])
tag_scores shape: torch.Size([18, 47])
tag_scores shape: torch.Size([26, 47])
tag_scores shape: torch.Size([22, 47])
tag_scores shape: torch.Size([23, 47])
tag_scores shape: torch.Size([41, 47])
tag_scores shape: torch.Size([21, 47])
tag_scores shape: torch.Size([39, 47])
tag_scores shape: torch.Size([34, 47])
tag_scores shape: torch.Size([28, 47])
tag_scores shape: torch.Size([34, 47])
tag_scores shape: torch.Size([44, 47])
tag_scores shape: torch.Size([17, 47])
tag_scores shape: torch.Size([13, 47])
tag_scores shape: torch.Size([29, 47])
tag_scores shape: torch.Size([42, 47])
tag_scores shape: torch.Size([9, 47])
tag_scores shape: torch.Size([8, 47])
tag_scores shape: torch.Size([35, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([15, 47])
tag_scores shape: torch.Siz

tag_scores shape: torch.Size([23, 47])
tag_scores shape: torch.Size([14, 47])
tag_scores shape: torch.Size([7, 47])
tag_scores shape: torch.Size([8, 47])
tag_scores shape: torch.Size([19, 47])
tag_scores shape: torch.Size([27, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([16, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([16, 47])
tag_scores shape: torch.Size([31, 47])
tag_scores shape: torch.Size([15, 47])
tag_scores shape: torch.Size([18, 47])
tag_scores shape: torch.Size([22, 47])
tag_scores shape: torch.Size([29, 47])
tag_scores shape: torch.Size([27, 47])
tag_scores shape: torch.Size([34, 47])
tag_scores shape: torch.Size([38, 47])
tag_scores shape: torch.Size([43, 47])
tag_scores shape: torch.Size([26, 47])
tag_scores shape: torch.Size([13, 47])
tag_scores shape: torch.Size([8, 47])
tag_scores shape: torch.Size([10, 47])
tag_scores shape: torch.Size([15, 47])
tag_scores shape: torch.Size([24, 47])
tag_scores shape: torch.Size

...

tag_scores shape: torch.Size([20, 47])
tag_scores shape: torch.Size([28, 47])
tag_scores shape: torch.Size([17, 47])
tag_scores shape: torch.Size([33, 47])
tag_scores shape: torch.Size([20, 47])
tag_scores shape: torch.Size([13, 47])
tag_scores shape: torch.Size([23, 47])
tag_scores shape: torch.Size([35, 47])
tag_scores shape: torch.Size([36, 47])
tag_scores shape: torch.Size([16, 47])
tag_scores shape: torch.Size([11, 47])
tag_scores shape: torch.Size([6, 47])
tag_scores shape: torch.Size([10, 47])
tag_scores shape: torch.Size([20, 47])
tag_scores shape: torch.Size([29, 47])
tag_scores shape: torch.Size([14, 47])
tag_scores shape: torch.Size([19, 47])
tag_scores shape: torch.Size([21, 47])
tag_scores shape: torch.Size([11, 47])
tag_scores shape: torch.Size([37, 47])
tag_scores shape: torch.Size([34, 47])
tag_scores shape: torch.Size([10, 47])
tag_scores shape: torch.Size([36, 47])
tag_scores shape: torch.Size([32, 47])
tag_scores shape: torch.Size([26, 47])
tag_scores shape: torch.Si

tag_scores shape: torch.Size([27, 47])
tag_scores shape: torch.Size([27, 47])
tag_scores shape: torch.Size([39, 47])
tag_scores shape: torch.Size([26, 47])
tag_scores shape: torch.Size([23, 47])
tag_scores shape: torch.Size([11, 47])
tag_scores shape: torch.Size([51, 47])
tag_scores shape: torch.Size([24, 47])
tag_scores shape: torch.Size([29, 47])
tag_scores shape: torch.Size([19, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([21, 47])
tag_scores shape: torch.Size([35, 47])
tag_scores shape: torch.Size([13, 47])
tag_scores shape: torch.Size([9, 47])
tag_scores shape: torch.Size([15, 47])
tag_scores shape: torch.Size([41, 47])
tag_scores shape: torch.Size([21, 47])
tag_scores shape: torch.Size([11, 47])
tag_scores shape: torch.Size([24, 47])
tag_scores shape: torch.Size([24, 47])
tag_scores shape: torch.Size([28, 47])
tag_scores shape: torch.Size([26, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([24, 47])
tag_scores shape: torch.Si

tag_scores shape: torch.Size([22, 47])
tag_scores shape: torch.Size([21, 47])
tag_scores shape: torch.Size([50, 47])
tag_scores shape: torch.Size([31, 47])
tag_scores shape: torch.Size([14, 47])
tag_scores shape: torch.Size([30, 47])
tag_scores shape: torch.Size([30, 47])
tag_scores shape: torch.Size([14, 47])
tag_scores shape: torch.Size([15, 47])
tag_scores shape: torch.Size([18, 47])
tag_scores shape: torch.Size([17, 47])
tag_scores shape: torch.Size([9, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([20, 47])
tag_scores shape: torch.Size([49, 47])
tag_scores shape: torch.Size([14, 47])
tag_scores shape: torch.Size([22, 47])
tag_scores shape: torch.Size([55, 47])
tag_scores shape: torch.Size([11, 47])
tag_scores shape: torch.Size([23, 47])
tag_scores shape: torch.Size([44, 47])
tag_scores shape: torch.Size([13, 47])
tag_scores shape: torch.Size([14, 47])
tag_scores shape: torch.Size([29, 47])
tag_scores shape: torch.Size([45, 47])
tag_scores shape: torch.Si

tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([8, 47])
tag_scores shape: torch.Size([30, 47])
tag_scores shape: torch.Size([24, 47])
tag_scores shape: torch.Size([22, 47])
tag_scores shape: torch.Size([19, 47])
tag_scores shape: torch.Size([27, 47])
tag_scores shape: torch.Size([31, 47])
tag_scores shape: torch.Size([34, 47])
tag_scores shape: torch.Size([12, 47])
tag_scores shape: torch.Size([29, 47])
tag_scores shape: torch.Size([29, 47])
tag_scores shape: torch.Size([36, 47])
tag_scores shape: torch.Size([16, 47])
tag_scores shape: torch.Size([17, 47])
tag_scores shape: torch.Size([17, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([15, 47])
tag_scores shape: torch.Size([25, 47])
tag_scores shape: torch.Size([28, 47])
tag_scores shape: torch.Size([27, 47])
tag_scores shape: torch.Size([30, 47])
tag_scores shape: torch.Size([30, 47])
tag_scores shape: torch.Size([37, 47])
tag_scores shape: torch.Size([21, 47])
tag_scores shape: torch.Si

KeyboardInterrupt: 